
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

Establish the IAM role that Glue will be using. This role needs to include the policies stated in this link: https://docs.aws.amazon.com/glue/latest/ug/notebook-getting-started.html
This step can be avoided if you already chose the IAM role during the set up of the Glue notebook session. Change the role name to yours.

In [ ]:
%iam_role arn:aws:iam::930929117161:role/<your-glue-role>

Current iam_role is arn:aws:iam::930929117161:role/glue-connection-role
iam_role has been set to arn:aws:iam::930929117161:role/glue-connection-role.


Establish the glue version in 3.0:

In [2]:
%glue_version 3.0

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Setting Glue version to: 3.0


Establish the connection the Glue job will be using to connect to oracle. Change the name to your connection:

In [4]:
%connections <your-connection-name>

Connections to be included:
oracle-connection


Include the necessary libraries:

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::930929117161:role/glue-connection-role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: b3aaaffe-b4e4-4684-b335-ac8ff89881ef
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session b3aaaffe-b4e4-4684-b335-ac8ff89881ef to get into ready status...
Session b3aaaffe-b4e4-4684-b335-ac8ff89881ef has been created.



Define a method to connect to Oracle through a Dynamicframe:

In [2]:
def directJDBCSource(
    glueContext,
    connectionName,
    connectionType,
    database,
    table,
    redshiftTmpDir,
    transformation_ctx,
) -> DynamicFrame:

    connection_options = {
        "useConnectionProperties": "true",
        "dbtable": table,
        "connectionName": connectionName,
    }

    if redshiftTmpDir:
        connection_options["redshiftTmpDir"] = redshiftTmpDir

    return glueContext.create_dynamic_frame.from_options(
        connection_type=connectionType,
        connection_options=connection_options,
        transformation_ctx=transformation_ctx,
    )

Create the Spark and Glue contexts:

In [3]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Check the session number and status:

In [6]:
%list_sessions

The first 1 sessions are:
b3aaaffe-b4e4-4684-b335-ac8ff89881ef


In [8]:
%status

Session ID: b3aaaffe-b4e4-4684-b335-ac8ff89881ef
Status: READY
Role: arn:aws:iam::930929117161:role/glue-connection-role
CreatedOn: 2022-12-22 16:33:13.156000+00:00
GlueVersion: 3.0
Worker Type: G.1X
Number of Workers: 5
Region: us-east-1
Connections: ['oracle-connection']
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Arguments Passed: ['--glue_kernel_version: 0.37.0', '--enable-glue-datacatalog: true']


Connect to Oracle using the method defined earlier and extract the defined table into a Glue Dynamicframe. Change the parameters to yours:

In [4]:
# Script generated for node Oracle SQL
OracleSQL_node1671594088379 = directJDBCSource(
    glueContext,
    connectionName="<your-oracle-connection>",
    connectionType="oracle",
    database="<your-oracle-database-name>",
    table="<your-table-name>",
    redshiftTmpDir="",
    transformation_ctx="OracleSQL_node1671594088379",
)

Convert the Dynamicframe to a Dataframe, print its schema, and show some results in order to understand data types and debbug some problems:

In [12]:
Dataframe = OracleSQL_node1671594088379.toDF().printSchema()

root
 |-- ID: decimal(38,10) (nullable = true)
 |-- NAME: string (nullable = true)
 |-- ABBREVIATED_NAME: string (nullable = true)
 |-- HOME_FIELD_ID: decimal(3,0) (nullable = true)
 |-- SPORT_TYPE_NAME: string (nullable = true)
 |-- SPORT_LEAGUE_SHORT_NAME: string (nullable = true)
 |-- SPORT_DIVISION_SHORT_NAME: string (nullable = true)


In [9]:
Dataframe.show()

+--------------+--------------------+----------------+-------------+---------------+-----------------------+-------------------------+
|            ID|                NAME|ABBREVIATED_NAME|HOME_FIELD_ID|SPORT_TYPE_NAME|SPORT_LEAGUE_SHORT_NAME|SPORT_DIVISION_SHORT_NAME|
+--------------+--------------------+----------------+-------------+---------------+-----------------------+-------------------------+
|511.0000000000|   Carolina Panthers|             CAR|           40|       football|                    NFL|                NFC South|
|121.0000000000| St. Louis Cardinals|             STL|            3|       baseball|                    MLB|               NL Central|
| 31.0000000000|   Oakland Athletics|             OAK|           18|       baseball|                    MLB|                  AL West|
|261.0000000000|San Francisco Giants|              SF|            2|       baseball|                    MLB|                  NL West|
|111.0000000000|    San Diego Padres|              SD| 

Change some data types of the initial Dynamicframe with the ApplyMapping transformation. Here you need to change the columns and data types according to your case. The provided ones are part of the example:

In [10]:
# Script generated for node ApplyMapping
ApplyMapping_node2 = ApplyMapping.apply(
    frame=OracleSQL_node1671594088379,
    mappings=[
        ("abbreviated_name", "string", "abbreviated_name", "string"),
        ("sport_type_name", "string", "sport_type_name", "string"),
        ("sport_division_short_name", "string", "sport_division_short_name", "string"),
        ("id", "decimal", "id", "int"),
        ("home_field_id", "decimal", "home_field_id", "int"),
        ("sport_league_short_name", "string", "sport_league_short_name", "string"),
        ("name", "string", "name", "string"),
    ],
    transformation_ctx="ApplyMapping_node2",
)

Write the resulting Dynamicframe into S3. You need to change the path, catalogDatabase, catalogTableName, and format:

In [11]:
# Script generated for node S3 bucket
S3bucket_node3 = glueContext.getSink(
    path="s3://<your-bucket>/oracle-notebook/",
    connection_type="s3",
    updateBehavior="UPDATE_IN_DATABASE",
    partitionKeys=[],
    enableUpdateCatalog=True,
    transformation_ctx="S3bucket_node3",
)
S3bucket_node3.setCatalogInfo(catalogDatabase="data-lake", catalogTableName="oracle")
S3bucket_node3.setFormat("json")
S3bucket_node3.writeFrame(ApplyMapping_node2)